### Pranay Sharma

In [ ]:
import tensorflow as tf
from debugpy import connect
from matplotlib import axis
from os import name



In [ ]:
def downsample(x,filters,name):
  x=tf.keras.layers.Conv2D(filters=filters,kernel_size=(2,2),padding = 'same',name = f'{name}_CONV2D')(x)
  x=tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding = 'same',name = f'{name}_MAXPOOLING2D')(x)
  x=tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9,name = f'{name}_BN')(x)
  x=tf.keras.layers.Activation(activation='relu',name = f'{name}_RELU')(x)
  return x

In [ ]:
def upsampling(x,resudial_x,filters,name):
  x=tf.keras.layers.Conv2D(filters=filters,kernel_size=(2,2),padding='same',name = f'{name}_CONV2D')(x)
  x=tf.keras.layers.UpSampling2D(size=(2,2),name = f'{name}_US')(x)
  x=tf.keras.layers.Add(name = f'{name}_ADD')([x,resudial_x])
  x=tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9,name = f'{name}_BN')(x)
  x=tf.keras.layers.Activation(activation='relu',name = f'{name}_RELU')(x)
  return x


In [ ]:
def DSPP(inputs, filters, spatial_shape):
    conv1 = tf.keras.layers.Conv2D(
        filters=filters, kernel_size=3, dilation_rate=6, padding="same",name=f"DSPP_conv1_{filters}"
    )(inputs)
    conv2 = tf.keras.layers.Conv2D(
        filters=filters, kernel_size=3, dilation_rate=12, padding="same",name=f"DSPP_conv2_{filters}"
    )(inputs)
    conv3 = tf.keras.layers.Conv2D(
        filters=filters, kernel_size=3, dilation_rate=18, padding="same",name=f"DSPP_conv3_{filters}"
    )(inputs)
    Global2D = tf.keras.layers.GlobalAveragePooling2D(name=f"DSPP_global2d_{filters}")(inputs)
    x = tf.keras.layers.Reshape((1, 1, -1),name=f"reshape1_{filters}")(Global2D)
    x = tf.keras.layers.UpSampling2D(size=spatial_shape,name=f"upsampling2d_{filters}")(x)
    x = tf.keras.layers.Concatenate(axis=-1,name=f"concat_{filters}")([conv1, conv2, conv3, x])
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, padding="same",name=f"DSPP_conv_final_{filters}")(x)
    x = tf.keras.layers.BatchNormalization(epsilon=1e-5, momentum=0.9,name=f"DSPP_bn_{filters}")(x)
    x = tf.keras.layers.Activation(activation='relu',name=f"DSPP_relu_{filters}")(x)
    return x


In [ ]:
def UNet(name = 'UNet'):
  inputs=tf.keras.layers.Input(shape=(256,256,3),name = f'{name}_input')
  d1  = downsample(inputs,16,name = f'{name}_d1') #128
  d2  = downsample(d1,32,name = f'{name}_d2') #64
  d3  = downsample(d2,64,name = f'{name}_d3') #32
  d4  = downsample(d3,128,name = f'{name}_bottleneck') #16
  u3 = upsampling(d4,d3,64,name = f'{name}_u3') #32
  u2 = upsampling(u3,d2,32,name = f'{name}_u2') #64
  u1 = upsampling(u2,d1,16,name = f'{name}_u1') #128
  dspp = DSPP(inputs=u2,filters=32,spatial_shape= (64, 64))
  dspp = tf.keras.layers.UpSampling2D(size=(4,4),name= f'{name}_dspp_Upsampling')(dspp)
  u0 = tf.keras.layers.UpSampling2D(size=(2,2),name = f'{name}_Upsampling')(u1) 
  u0 = tf.keras.layers.Concatenate(axis=-1)([u0,dspp]) 

  outputs = tf.keras.layers.Conv2D(filters=3,kernel_size=(1,1),activation = 'softmax',padding='same',name = f'{name}_outputs')(u0)
  model = tf.keras.models.Model(inputs=inputs,outputs=outputs,name=name)
  return model



In [ ]:
model = UNet()

In [ ]:
model.summary()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
image_path = "images/GumsTeethDataSet/OriginalImages/0.jpg"


# Load and preprocess the image
img = image.load_img(image_path, target_size=(256, 256))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # normalize to [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

# Run the image through the model
prediction = model.predict(img_array)

In [ ]:
import matplotlib.pyplot as plt
# Display the original image and the prediction
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img_array[0])
plt.title("Original Image")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(prediction[0, :, :, 0], cmap='gray')
plt.title("Prediction")
plt.axis("off") 
plt.show()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import os
def mask_read_input(folder_path,numberof_datapoint=30):
  images = []
  folder_names = os.listdir(folder_path)
  folder_names= folder_names[:numberof_datapoint]
  folder_names.sort()
  for folder in folder_names:
    img_path_1 = os.path.join(folder_path, folder, 'Teeth.png')
    img_1 = tf.io.read_file(img_path_1)
    img_1 = tf.image.decode_jpeg(img_1, channels=1)
    img_1 = tf.image.resize_with_pad(
            img_1,
            target_height=256,
            target_width=256,
            method=tf.image.ResizeMethod.BILINEAR,
            antialias=False
          )/255
    img_path_2 = os.path.join(folder_path, folder, 'Gums.png')
    img_2 = tf.io.read_file(img_path_2)
    img_2 = tf.image.decode_jpeg(img_2, channels=1)
    img_2 = tf.image.resize_with_pad(
            img_2,
            target_height=256,
            target_width=256,
            method=tf.image.ResizeMethod.BILINEAR,
            antialias=False
          )/255
    img_3=tf.nn.relu(1.0-(img_1+img_2))
    img = tf.concat([img_1, img_2,img_3], axis=-1)
    images.append(img)
  images_batch = tf.stack(images)
  return images_batch

In [ ]:
import os
def image_read_input(folder_path,numberof_datapoint=30):
  images = []
  filenames = os.listdir(folder_path)
  filenames= filenames[:numberof_datapoint]
  filenames.sort()
  for filename in filenames:
    img_path = os.path.join(folder_path, filename)
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(
            img,
            target_height=256,
            target_width=256,
            method=tf.image.ResizeMethod.BILINEAR,
            antialias=False
          )/255
    images.append(img)
  images_batch = tf.stack(images)
  return images_batch

In [ ]:
batch_mask = mask_read_input("images/GumsTeethDataSet/Masks")
batch_mask

In [ ]:
batch = image_read_input("images/GumsTeethDataSet/OriginalImages")
batch

In [ ]:
matrices=model.fit(batch, batch_mask, epochs=50, batch_size=4)